In [7]:
#Modules imported
%pip install kagglehub tensorflow pandas numpy scikit-learn ast

import kagglehub
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
import ast

  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 35, in <module>
        File "C:\Users\User\AppData\Local\Temp\pip-install-7hm2z5hr\ast_ea555b8cbacd45cbaa346db298608c5e\setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen codecs>", line 918, in open
      FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-7hm2z5hr\\ast_ea555b8cbacd45cbaa346db298608c5e\\AST/README'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating

In [8]:
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

# Load datasets
metadata = pd.read_csv(path + "/movies_metadata.csv", low_memory=False)
credits = pd.read_csv(path + "/credits.csv")
keywords = pd.read_csv(path + "/keywords.csv")

# Make sure IDs match
metadata["id"] = metadata["id"].astype(str)
credits["id"] = credits["id"].astype(str)
keywords["id"] = keywords["id"].astype(str)

# Merge into one dataset
movies = metadata.merge(credits, on="id").merge(keywords, on="id")

In [9]:

def parse_features(x):
    try:
        return [d['name'] for d in ast.literal_eval(x)]
    except:
        return []

def get_director(x):
    try:
        crew = ast.literal_eval(x)
        for d in crew:
            if d['job'] == 'Director':
                return d['name']
        return ''
    except:
        return ''

# Extract features
movies['genres'] = movies['genres'].apply(parse_features)
movies['keywords'] = movies['keywords'].apply(parse_features)
movies['cast'] = movies['cast'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)[:3]] if pd.notna(x) else [])
movies['director'] = movies['crew'].apply(get_director)

# Combine into one string
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['director'].apply(lambda x: [x])
movies['tags'] = movies['tags'].apply(lambda x: ' '.join(x))


In [10]:
# Convert tags to vectors
cv = CountVectorizer(max_features=5000, stop_words='english')
vector_matrix = cv.fit_transform(movies['tags'].fillna(''))

# Fit KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(vector_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
def recommend_movies(movie_name, n_recommendations=5):
    if movie_name not in movies['title'].values:
        return "Movie not found in the dataset."
    
    idx = movies[movies['title'] == movie_name].index[0]
    distances, indices = knn.kneighbors(vector_matrix[idx], n_neighbors=n_recommendations + 1)
    
    recommended_movies = []
    for i in range(1, len(distances[0])):
        recommended_movies.append(movies.iloc[indices[0][i]]['title'])
    
    return recommended_movies

print(recommend_movies("The Dark Knight", 5))

['Batman Begins', 'The Dark Knight Rises', 'Batman & Robin', 'Petos', 'Kaakha Kaakha']
